# We start off by importing the data

In [51]:
import math
import pandas as pd
import shared
pd.reset_option('precision')
#pd.set_option('float_format', '{:f}'.format)
pd.set_option('float_format', '{:,.1f}'.format)

bl_kurzel = shared.bl_kurzel

dfs = {}
for kurzel in shared.bl_kurzel:
    dfs[kurzel] = pd.read_csv(f'data/vac_{kurzel}.csv', sep=',', index_col=0, parse_dates=True)
    # it's important to define index and parse_dates



In [52]:
list(dfs['BY'].columns)

['publication_date',
 'dosen_kumulativ',
 'personen_erst_kumulativ',
 'personen_voll_kumulativ',
 'personen_min1_kumulativ',
 'personen_zweit_kumulativ',
 'personen_auffr_kumulativ',
 'dosen_biontech_kumulativ',
 'personen_erst_biontech_kumulativ',
 'personen_voll_biontech_kumulativ',
 'personen_min1_biontech_kumulativ',
 'personen_zweit_biontech_kumulativ',
 'personen_auffr_biontech_kumulativ',
 'dosen_moderna_kumulativ',
 'personen_erst_moderna_kumulativ',
 'personen_voll_moderna_kumulativ',
 'personen_min1_moderna_kumulativ',
 'personen_zweit_moderna_kumulativ',
 'personen_auffr_moderna_kumulativ',
 'dosen_astrazeneca_kumulativ',
 'personen_erst_astrazeneca_kumulativ',
 'personen_voll_astrazeneca_kumulativ',
 'personen_min1_astrazeneca_kumulativ',
 'personen_zweit_astrazeneca_kumulativ',
 'personen_auffr_astrazeneca_kumulativ',
 'dosen_janssen_kumulativ',
 'personen_erst_janssen_kumulativ',
 'personen_voll_janssen_kumulativ',
 'personen_min1_janssen_kumulativ',
 'personen_zweit_jans

### Let' pick the few columns that interest us

In [53]:
for kurzel in bl_kurzel:
    dfs[kurzel] = dfs[kurzel][['publication_date', 'dosen_kumulativ', 'personen_erst_kumulativ', 'personen_voll_kumulativ']]

# Let's fix the data

## From April 28th the data is just wrong. I'll make a cutoff there. it isn't relevant anyway.

In [54]:
for kurzel in bl_kurzel:
    print(len(dfs[kurzel].index))

378
378
378
378
378
378
378
378
378
378
378
378
378
378
378
378
378


In [55]:
length = len(dfs['BY'].index)
for kurzel in bl_kurzel:
    dfs[kurzel] = dfs[kurzel].head(length - 19)

In [56]:
dfs['BY'].tail(5)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ
date,,,,
2022-04-11,2022-04-12 08:00,26471410,"9,227,229.0",9834378
2022-04-12,2022-04-13 08:00,26476263,"9,227,618.0",9835329
2022-04-13,2022-04-14 08:00,26480745,"9,227,996.0",9836261
2022-04-18,2022-04-19 08:00,26487269,"9,228,591.0",9837826
2022-04-19,2022-04-20 08:00,26490860,"9,228,933.0",9838665


### There are two spikes, those are just wrong data, let's fix them manually

In [57]:
dfs['BY'].dosen_kumulativ = dfs['BY'].dosen_kumulativ.replace(384311, 343668)
dfs['BY'].dosen_kumulativ = dfs['BY'].dosen_kumulativ.replace(1150129, 1186228)




### New column shots_today instead of total doses til now (dosen_kumulativ)

Each row is seen as it's own entry, the value of a day should not depend on the previous value.
But that is the case for the dosen_kumulativ column.
Instead we will add a new column 'heutige_dosen' and recreate the dosen_kumulativ column

In [58]:
dfs['BY'].head(5)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ
date,,,,
2020-12-27,2020-12-28 16:15,3389,"3,389.0",0
2020-12-28,2020-12-29 08:00,5219,"5,219.0",0
2020-12-30,2020-12-31 08:30,28206,"28,206.0",0
2020-12-31,2021-01-01 12:30,37955,"37,955.0",0
2021-01-01,2021-01-02 08:00,39005,"39,005.0",0


In [59]:
def add_dif_column(df):
    df['shots_today'] = df.dosen_kumulativ - df.dosen_kumulativ.shift(1)
    df['shots_today'] = df['shots_today'].fillna(0)
    return df.astype({'shots_today': 'int64'})

for kurzel in bl_kurzel:
    dfs[kurzel] = add_dif_column(dfs[kurzel])

### Add missing rows with empty values

In [60]:
dfs['DE'].head(10)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today
date,,,,,
2020-12-27,2020-12-28 16:15,21566,"21,566.0",0,0
2020-12-28,2020-12-29 08:00,41962,"41,962.0",0,20396
2020-12-30,2020-12-31 08:30,131626,"131,626.0",0,89664
2020-12-31,2021-01-01 12:30,165575,"165,575.0",0,33949
2021-01-01,2021-01-02 08:00,188553,"188,553.0",0,22978
2021-01-02,2021-01-03 08:00,238809,"238,809.0",0,50256
2021-01-03,2021-01-04 12:00,265986,"265,986.0",0,27177
2021-01-04,2021-01-05 12:00,316962,"316,962.0",0,50976
2021-01-05,2021-01-06 11:00,367331,"367,331.0",0,50369


In [61]:
dfs['DE'].tail(10)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today
date,,,,,
2022-04-04,2022-04-05 08:00,172175901,"60,081,679.0",63193985,32805
2022-04-05,2022-04-06 08:00,172224482,"60,086,537.0",63203579,48581
2022-04-06,2022-04-07 08:00,172279700,"60,092,314.0",63215379,55218
2022-04-07,2022-04-08 08:00,172332148,"60,096,699.0",63225075,52448
2022-04-10,2022-04-11 08:00,172395880,"60,103,214.0",63238599,63732
2022-04-11,2022-04-12 08:00,172420421,"60,105,540.0",63243601,24541
2022-04-12,2022-04-13 08:00,172459530,"60,109,484.0",63250855,39109
2022-04-13,2022-04-14 08:00,172495179,"60,114,114.0",63256538,35649
2022-04-18,2022-04-19 08:00,172536857,"60,118,591.0",63265523,41678


In [62]:
def fix_missing_days(df):
    # fill in the dates
    idx = pd.date_range(start='2020-12-26', end=df.index.max())

    df = df.reindex(idx)

    # Create new index column because that's waay easier than having the date column be the index
    df = df.reset_index()
    df.at[0,'dosen_kumulativ'] = 0
    df.at[0,'shots_today'] = 0
    df.at[0,'personen_erst_kumulativ'] = 0
    df.at[0,'personen_voll_kumulativ'] = 0
    df.at[1,'shots_today'] =  df.iloc[1, :]['dosen_kumulativ']

    return df.rename(columns={'index': 'date'})

for kurzel in bl_kurzel:
    dfs[kurzel] = fix_missing_days(dfs[kurzel])


### Fix NaN values in shots_today

In [63]:
def fix_NaN_dosen(df, column):
    i = 0
    while i < len(df.index):
        row = df.iloc[i, :]
        if pd.isnull(row[column]):
            j = 1
            new_row = df.iloc[i+j, :]
            while pd.isnull(new_row[column]):
                j = j + 1
                new_row = df.iloc[i+j, :]
            next_valid_row = df.iloc[i+j, :]
            quotient = next_valid_row[column] / (j+1)
            df.at[i+j,column] = quotient
            for to_change in range(i, i+j):
                df.at[to_change,column] = quotient
            i = i+j
        else:
            i = i + 1

for kurzel in bl_kurzel:
    fix_NaN_dosen(dfs[kurzel], 'shots_today')



### New shots_sum column

In [64]:
def add_shots_sum(df):
    df['shots_sum'] = 0
    df['shots_sum'] = df['shots_today'].cumsum().round()
    df.shots_sum = df.shots_sum.astype(int)
    return df

for kurzel in bl_kurzel:
    dfs[kurzel] = add_shots_sum(dfs[kurzel])

In [65]:
dfs['BY'].head(6)

,date,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today,shots_sum
0,2020-12-26,NaN,0.0,0.0,0.0,0.0,0
1,2020-12-27,2020-12-28 16:15,"3,389.0","3,389.0",0.0,"3,389.0",3389
2,2020-12-28,2020-12-29 08:00,"5,219.0","5,219.0",0.0,"1,830.0",5219
3,2020-12-29,NaN,NaN,NaN,NaN,"11,493.5",16712
4,2020-12-30,2020-12-31 08:30,"28,206.0","28,206.0",0.0,"11,493.5",28206
5,2020-12-31,2021-01-01 12:30,"37,955.0","37,955.0",0.0,"9,749.0",37955


### New column: Day of the week

In [66]:
for kurzel in bl_kurzel:
    dfs[kurzel] = shared.add_weekday_stuff(dfs[kurzel], 'date')

## Add columns 'per 100k Pop'

In [67]:
def add_pop_relative_stuff(kurzel):
    pop = shared.get_land_pop(kurzel)
    dfs[kurzel]['shots_sum_100k'] = dfs[kurzel]['shots_sum'] / pop * 100_000
    dfs[kurzel]['shots_today_100k'] = dfs[kurzel]['shots_today'] / pop * 100_000



for kurzel in bl_kurzel:
    add_pop_relative_stuff(kurzel)

## Let's take a look

In [68]:
dfs['DE'].head(15)

,date,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today,shots_sum,is_weekend,weekday_name,calendar_week,year_and_week,shots_sum_100k,shots_today_100k
0,2020-12-26,NaN,0.0,0.0,0.0,0.0,0,True,Sat,52,2020_52,0.0,0.0
1,2020-12-27,2020-12-28 16:15,"21,566.0","21,566.0",0.0,"21,566.0",21566,True,Sun,52,2020_52,25.9,25.9
2,2020-12-28,2020-12-29 08:00,"41,962.0","41,962.0",0.0,"20,396.0",41962,False,Mon,53,2020_53,50.4,24.5
3,2020-12-29,NaN,NaN,NaN,NaN,"44,832.0",86794,False,Tue,53,2020_53,104.3,53.9
4,2020-12-30,2020-12-31 08:30,"131,626.0","131,626.0",0.0,"44,832.0",131626,False,Wed,53,2020_53,158.2,53.9
5,2020-12-31,2021-01-01 12:30,"165,575.0","165,575.0",0.0,"33,949.0",165575,False,Thu,53,2020_53,199.0,40.8
6,2021-01-01,2021-01-02 08:00,"188,553.0","188,553.0",0.0,"22,978.0",188553,False,Fri,53,2020_53,226.7,27.6
7,2021-01-02,2021-01-03 08:00,"238,809.0","238,809.0",0.0,"50,256.0",238809,True,Sat,53,2020_53,287.1,60.4
8,2021-01-03,2021-01-04 12:00,"265,986.0","265,986.0",0.0,"27,177.0",265986,True,Sun,53,2020_53,319.7,32.7
9,2021-01-04,2021-01-05 12:00,"316,962.0","316,962.0",0.0,"50,976.0",316962,False,Mon,1,2021_01,381.0,61.3


In [69]:
dfs['DE'].tail(10)


,date,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today,shots_sum,is_weekend,weekday_name,calendar_week,year_and_week,shots_sum_100k,shots_today_100k
470,2022-04-10,2022-04-11 08:00,"172,395,880.0","60,103,214.0","63,238,599.0","21,244.0",172395880,True,Sun,14,2022_14,"207,230.1",25.5
471,2022-04-11,2022-04-12 08:00,"172,420,421.0","60,105,540.0","63,243,601.0","24,541.0",172420421,False,Mon,15,2022_15,"207,259.6",29.5
472,2022-04-12,2022-04-13 08:00,"172,459,530.0","60,109,484.0","63,250,855.0","39,109.0",172459530,False,Tue,15,2022_15,"207,306.6",47.0
473,2022-04-13,2022-04-14 08:00,"172,495,179.0","60,114,114.0","63,256,538.0","35,649.0",172495179,False,Wed,15,2022_15,"207,349.5",42.9
474,2022-04-14,NaN,NaN,NaN,NaN,"8,335.6",172503515,False,Thu,15,2022_15,"207,359.5",10.0
475,2022-04-15,NaN,NaN,NaN,NaN,"8,335.6",172511850,False,Fri,15,2022_15,"207,369.5",10.0
476,2022-04-16,NaN,NaN,NaN,NaN,"8,335.6",172520186,True,Sat,15,2022_15,"207,379.5",10.0
477,2022-04-17,NaN,NaN,NaN,NaN,"8,335.6",172528521,True,Sun,15,2022_15,"207,389.6",10.0
478,2022-04-18,2022-04-19 08:00,"172,536,857.0","60,118,591.0","63,265,523.0","8,335.6",172536857,False,Mon,16,2022_16,"207,399.6",10.0
479,2022-04-19,2022-04-20 08:00,"172,566,113.0","60,123,492.0","63,270,874.0","29,256.0",172566113,False,Tue,16,2022_16,"207,434.7",35.2


## Save to pickle

In [70]:
from datetime import datetime
print(f"{datetime.now()}")

for kurzel in bl_kurzel:
    path = f'data/df_vac_{kurzel}.pkl'
    dfs[kurzel].to_pickle(path)
    print(f'Saved pickle to {path}')


2022-05-17 23:32:35.020980
Saved pickle to data/df_vac_BB.pkl
Saved pickle to data/df_vac_BE.pkl
Saved pickle to data/df_vac_BW.pkl
Saved pickle to data/df_vac_BY.pkl
Saved pickle to data/df_vac_DE.pkl
Saved pickle to data/df_vac_HB.pkl
Saved pickle to data/df_vac_HE.pkl
Saved pickle to data/df_vac_HH.pkl
Saved pickle to data/df_vac_MV.pkl
Saved pickle to data/df_vac_NI.pkl
Saved pickle to data/df_vac_NW.pkl
Saved pickle to data/df_vac_RP.pkl
Saved pickle to data/df_vac_SH.pkl
Saved pickle to data/df_vac_SL.pkl
Saved pickle to data/df_vac_SN.pkl
Saved pickle to data/df_vac_ST.pkl
Saved pickle to data/df_vac_TH.pkl
